# HW 4 - Ishaan Sathaye

## Section A: Derivations

In the following assume we have *p* predictors in our model, where *p* may be larger than 1, and *n* observations.

Hint: You may want to use the notation: $sign(a) = {1 if a > 0, -1 if a < 0}$

1. Give the *gradient* equation for Ordinary Least Squares Regression.
    - loss function: $L(\beta) = \frac{1}{n} \sum_{i=1}^{n} (y_i - \beta^T x_i)^2$
    - $\nabla L(\beta) = \frac{d}{d\beta} \frac{1}{n} \sum_{i=1}^{n} (y_i - \beta^T x_i)^2$
    - $\nabla L(\beta) = \frac{-2}{n} \sum_{i=1}^{n} (y_i - \beta^T x_i) x_i$
    - In matrix form: $\nabla L(\beta) = \frac{-2}{n} X^T (Y - \hat{Y})$

2. Give the *gradient* equation for Ridge Regression.
    - loss function: $L(\beta) = \frac{1}{n} \sum_{i=1}^{n} (y_i - \beta^T x_i)^2 + \lambda \sum_{j=1}^{p} \beta_j^2$
    - $\nabla L(\beta) = \frac{d}{d\beta} \frac{1}{n} \sum_{i=1}^{n} (y_i - \beta^T x_i)^2 + \lambda \sum_{j=1}^{p} \beta_j^2$
    - $\nabla L(\beta) = \frac{-2}{n} \sum_{i=1}^{n} (y_i - \beta^T x_i) x_i + 2 \lambda \beta$
    - In matrix form: $\nabla L(\beta) = \frac{-2}{n} X^T (Y - \hat{Y}) + 2 \lambda \beta$

3. Give the *gradient* equation for Lasso Regression.
    - loss function: $L(\beta) = \frac{1}{n} \sum_{i=1}^{n} (y_i - \beta^T x_i)^2 + \lambda \sum_{j=1}^{p} |\beta_j|$
    - $\nabla L(\beta) = \frac{d}{d\beta} \frac{1}{n} \sum_{i=1}^{n} (y_i - \beta^T x_i)^2 + \lambda \sum_{j=1}^{p} |\beta_j|$
    - $\nabla L(\beta) = \frac{-2}{n} \sum_{i=1}^{n} (y_i - \beta^T x_i) x_i + \lambda sign(\beta)$
    - In matrix form: $\nabla L(\beta) = \frac{-2}{n} X^T (Y - \hat{Y}) + \lambda sign(\beta)$

4. Give the *gradient* equation for Linear Regression with a loss function of $L(\beta) = \sum_{i=1}^{n} (y_i - \hat{y}_i)^4 + \lambda \sum_{j=1}^{p} \beta_j^4$
    - $\nabla L(\beta) = -4 \sum_{i=1}^{n} (y_i - \hat{y}_i)^3 x_i + 4 \lambda \beta^3$
    - In matrix form: $\nabla L(\beta) = -4 X^T (Y - \hat{Y})^3 + 4 \lambda \beta^3$

5. Give the *gradient* equation for Linear Regression with a loss function of $L(\beta) = \sum_{i=1}^{n} |y_i - \hat{y}_i| + \lambda \sum_{j=1}^{p} |\beta_j|$
    - $\nabla L(\beta) = - \sum_{i=1}^{n} sign(y_i - \hat{y}_i) x_i + \lambda sign(\beta)$
    - In matrix form: $\nabla L(\beta) = - X^T sign(Y - \hat{Y}) + \lambda sign(\beta)$

## Section B: Coding

1. Write a function to implement gradient descent for LASSO estimation on the cannabis data from last week.

In [17]:
# function: fit_lasso
# inputs: Y, X, lambda, eta

#     initialize beta vector
    
#     while stopping condition is not met:
    
#         compute gradient values at current beta values
        
#         update beta = beta - eta*gradient
        
#         (optional) update eta
        
#         check for problematic iteration conditions
#         # too many iterations, too much time, or clues that 
# the gradient descent is not converging well.
        
#     return beta estimates


# Hint 1: Writing small helper functions, especially for 
# things like checking a stopping condition or checking the 
# iteration is converging, may help you cleanly keep track of 
# the steps in your code.

# Hint 2: While you may not hard code information about the 
# cannabis dataset into your function, you may tailor your 
# function so that it works well on this dataset specifically. 
# For example, you can choose your eta - and whether you keep 
# the value constant or change it at each iteration - to be a 
# value that happens to converge well on this particular dataset.

import numpy as np
import pandas as pd

def fit_lasso(Y, X, lambda_, eta, stop_condition=1e-2, decay=0.9, max_iter=100):
    beta = np.zeros(X.shape[1])
    eta_init = eta
    for i in range(max_iter):
        # compute gradient
        gradient = compute_gradient(Y, X, lambda_, beta)
        # exponential decay of learning rate
        eta = eta_init * (decay ** i)
        # update beta
        beta = beta - eta*gradient
        if check_stopping_condition(gradient, stop_condition):
            break
    return beta

def compute_gradient(Y, X, lambda_, beta):
    n = X.shape[0]
    return -2/n * X.T @ (Y - X @ beta) + lambda_ * np.sign(beta)

def check_stopping_condition(gradient, stop_condition):
    return np.linalg.norm(gradient) < stop_condition


2. Write a function to perform cross-validation on a set of lambdas.

In [18]:
def tune_lambda_split(train, test, lam_values, metric):
    X_train = train.drop(columns=['Rating']).values
    X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
    y_train = train['Rating'].values
    X_test = test.drop(columns=['Rating']).values
    X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))
    y_test = test['Rating'].values
    metrics = []
    for lam in lam_values:
        betas = fit_lasso(y_train, X_train, lam, 0.01, decay=0.75)
        y_pred = X_test @ betas
        if metric == 'r-sq':
            y_bar = np.mean(y_test)
            ss_tot = np.sum((y_test - y_bar) ** 2)
            ss_res = np.sum((y_test - y_pred) ** 2)
            r2 = 1 - ss_res / ss_tot
            metrics.append(r2)
        elif metric == 'mse':
            mse = np.mean((y_test - y_pred) ** 2)
            metrics.append(mse)
        elif metric == 'mae':
            mae = np.mean(np.abs(y_test - y_pred))
            metrics.append(mae)
    return pd.DataFrame({'lambda': lam_values, metric: metrics})

def tune_lambda(df, lam_values, metric, k):
    n = df.shape[0]
    fold_size = n // k
    metrics = []
    for lam in lam_values:
        metric_values = []
        for i in range(k):
            test = df.iloc[i * fold_size:(i + 1) * fold_size]
            train = df.drop(test.index)
            df_metric = tune_lambda_split(train, test, [lam], metric)
            metric_values.append(df_metric[metric].values[0])
        metrics.append(np.mean(metric_values))
    return pd.DataFrame({'lambda': lam_values, metric: metrics})

3. Apply your cross-validation function to the cannabis dataset to find the “best” lambda.

In [20]:
df = pd.read_csv("../hw3/cannabis_full.csv")
predictors = df.drop(columns=['Strain', 'Type', 'Effects', 'Flavor', 'Rating'])
df_clean = df.dropna(subset=predictors.columns)
df_clean = pd.get_dummies(df_clean, columns=['Type'], drop_first=True)
# standardize the data
predictors = df_clean.drop(columns=['Strain', 'Effects', 'Flavor', 'Rating'])
predictors = (predictors - predictors.mean()) / predictors.std()
predictors['Rating'] = df_clean['Rating']

lam_values = np.logspace(0, 5, num=100)
folds = 5
df_tune_rsq = tune_lambda(predictors, lam_values, 'r-sq', folds)
best_lambda_rsq = df_tune_rsq.loc[df_tune_rsq['r-sq'].idxmax()]['lambda']
print(best_lambda_rsq)

df_tune_mse = tune_lambda(predictors, lam_values, 'mse', folds)
best_lambda_mse = df_tune_mse.loc[df_tune_mse['mse'].idxmin()]['lambda']
print(best_lambda_mse)

df_tune_mae = tune_lambda(predictors, lam_values, 'mae', folds)
best_lambda_mae = df_tune_mae.loc[df_tune_mae['mae'].idxmin()]['lambda']
print(best_lambda_mae)

1.0
1.0
1.0


In [23]:
df_tune_rsq

,lambda,r-sq
0,1.000000,-24.951810
1,1.123324,-24.988616
2,1.261857,-25.030070
3,1.417474,-25.086174
4,1.592283,-25.150598
...,...,...
95,62802.914418,-28.851254
96,70548.023107,-28.851254
97,79248.289835,-28.851254
98,89021.508545,-28.851254


In [21]:
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split

X = predictors.drop(columns=['Rating']).values
y = predictors['Rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
lasso = LassoCV(alphas=lam_values, cv=5)
lasso.fit(X_train, y_train)

best_lambda_sklearn = lasso.alpha_
print(best_lambda_sklearn)

# eval on test set
test_score = lasso.score(X_test, y_test)
print(test_score)

100000.0
-9.80247941635426e-05
